# Phase Change Analysis

## Setup

In [1]:
!ls

README.md	    quick_start_pytorch.ipynb
circuits-over-time  quick_start_pytorch_images


In [2]:
%cd circuits-over-time

/notebooks/circuits-over-time


In [3]:
!pip install git+https://github.com/neelnanda-io/TransformerLens.git
!pip install circuitsvis
!pip install jaxtyping==0.2.13
!pip install einops
!pip install protobuf==3.20.*
!pip install plotly
!pip install torchtyping
!pip install git+https://github.com/neelnanda-io/neel-plotly.git

  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-0ydpp3df
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-0ydpp3df
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 0d2827ecce4ef17b86060bdaaaaf50e724684085
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 44.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 103.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 90.5 MB/s eta 0:00:00
  Created wheel for tra

In [3]:

IN_COLAB = False
from IPython import get_ipython

ipython = get_ipython()
# Code to automatically update the HookedTransformer code as its edited without restarting the kernel
ipython.magic("load_ext autoreload")
ipython.magic("autoreload 2")

/tmp/ipykernel_504/2182663555.py:6: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_504/2182663555.py:7: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [4]:
import os
import pathlib
from typing import List, Optional, Union
from collections import namedtuple

import torch
import numpy as np
import yaml

import einops
from fancy_einsum import einsum

from datasets import load_dataset
from transformers import pipeline
import plotly.io as pio
import plotly.express as px
#import pysvelte
from IPython.display import HTML

import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display

# if IN_COLAB or not DEBUG_MODE:
#     # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
#     pio.renderers.default = "colab"
# else:
#     pio.renderers.default = "plotly_mimetype+notebook"

if torch.cuda.is_available():
    device = int(os.environ.get("LOCAL_RANK", 0))
else:
    device = "cpu"

In [5]:
import transformers
from transformers import AutoConfig, AutoModel, AutoModelForCausalLM
import transformer_lens
import transformer_lens.utils as utils
import transformer_lens.patching as patching
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

from functools import partial

from torchtyping import TensorType as TT

In [6]:
from model_utils import (
    load_model,
    clear_gpu_memory
)
import circuit_utils as cu

In [7]:
torch.set_grad_enabled(False)

In [8]:
from neel_plotly import line, imshow, scatter

def l_imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def l_line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def l_scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

def two_lines(tensor1, tensor2, renderer=None, **kwargs):
    px.line(y=[utils.to_numpy(tensor1), utils.to_numpy(tensor2)], **kwargs).show(renderer)

## Analysis Functions

In [9]:
def get_induction_scores(model):
    seq_len = 100
    batch_size = 2

    prev_token_scores = torch.zeros((model.cfg.n_layers, model.cfg.n_heads), device="cuda")

    def prev_token_hook(pattern, hook):
        layer = hook.layer()
        diagonal = pattern.diagonal(offset=1, dim1=-1, dim2=-2)
        prev_token_scores[layer] = einops.reduce(diagonal, "batch head_index diagonal -> head_index", "mean")

    duplicate_token_scores = torch.zeros((model.cfg.n_layers, model.cfg.n_heads), device="cuda")

    def duplicate_token_hook(pattern, hook):
        layer = hook.layer()
        diagonal = pattern.diagonal(offset=seq_len, dim1=-1, dim2=-2)
        duplicate_token_scores[layer] = einops.reduce(diagonal, "batch head_index diagonal -> head_index", "mean")

    induction_scores = torch.zeros((model.cfg.n_layers, model.cfg.n_heads), device="cuda")

    def induction_hook(pattern, hook):
        layer = hook.layer()
        diagonal = pattern.diagonal(offset=seq_len-1, dim1=-1, dim2=-2)
        induction_scores[layer] = einops.reduce(diagonal, "batch head_index diagonal -> head_index", "mean")

    original_tokens = torch.randint(100, 20000, size=(batch_size, seq_len))
    repeated_tokens = einops.repeat(original_tokens, "batch seq_len -> batch (2 seq_len)").cuda()

    pattern_filter = lambda act_name: act_name.endswith("hook_pattern")
    loss = model.run_with_hooks(repeated_tokens, return_type="loss", fwd_hooks=[(pattern_filter, prev_token_hook), (pattern_filter, duplicate_token_hook), (pattern_filter, induction_hook)])

    return induction_scores, prev_token_scores, duplicate_token_scores

## Pythia Analysis

### Pythia 160M

In [17]:
model_name = "pythia-160m"

model_full_name = f"EleutherAI/{model_name}"

cache_dir="/media/curttigges/project-files/projects/circuits"

In [18]:
model = HookedTransformer.from_pretrained(
    model_full_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    #refactor_factored_attn_matrices=True,
    cache_dir=cache_dir,
)

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


In [19]:
from tqdm import tqdm

steps = [2**n for n in range(10)] + [n * 1000 for n in range(1, 10)] # + [n * 1000 for n in range(10, 143, 5)]
labels = [f"Step {n}" for n in steps]
induction_scores = torch.zeros((model.cfg.n_layers, model.cfg.n_heads, len(steps)), device="cuda")
induction_maxes = torch.zeros(len(steps))
induction_sums = torch.zeros(len(steps))

duplication_scores = torch.zeros((model.cfg.n_layers, model.cfg.n_heads, len(steps)), device="cuda")
duplication_maxes = torch.zeros(len(steps))
duplication_sums = torch.zeros(len(steps))

previous_token_scores = torch.zeros((model.cfg.n_layers, model.cfg.n_heads, len(steps)), device="cuda")
previous_token_maxes = torch.zeros(len(steps))
previous_token_sums = torch.zeros(len(steps))

for i in tqdm(range(len(steps))):
    model = HookedTransformer.from_pretrained(
        model_full_name,
        center_unembed=True,
        center_writing_weights=True,
        fold_ln=True,
        #refactor_factored_attn_matrices=True,
        revision=f"step{steps[i]}",
        cache_dir=cache_dir,
    )

    induction_scores[:,:,i], previous_token_scores[:,:,i], duplication_scores[:,:,i] = get_induction_scores(model)
    induction_maxes[i] = induction_scores[:,:,i].max()
    induction_sums[i] = induction_scores[:,:,i].sum()

  0%|          | 0/19 [00:00<?, ?it/s]

Using pad_token, but it is not set yet.
  5%|▌         | 1/19 [00:20<06:17, 21.00s/it]

Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


Using pad_token, but it is not set yet.
 11%|█         | 2/19 [00:44<06:17, 22.19s/it]

Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


...

Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


Using pad_token, but it is not set yet.
 95%|█████████▍| 18/19 [06:26<00:16, 16.40s/it]

Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


Using pad_token, but it is not set yet.
100%|██████████| 19/19 [06:49<00:00, 21.57s/it]

Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


In [20]:
l_line(induction_maxes, hover_name=labels, log_x=True)


In [21]:
l_line(induction_sums, hover_name=labels, log_x=True)

In [22]:
from circuit_utils import visualize_tensor
visualize_tensor(induction_scores.cpu(), labels, zmin=-1, zmax=1)

IntSlider(value=0, description='Slice:', max=18)

Output()

### Pythia 410M

In [8]:
model_name = "pythia-410m"

model_full_name = f"EleutherAI/{model_name}"

cache_dir="/media/curttigges/project-files/projects/circuits"

In [9]:
model = HookedTransformer.from_pretrained(
    model_full_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    #refactor_factored_attn_matrices=True,
    cache_dir=cache_dir,
)

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-410m into HookedTransformer


In [11]:
from tqdm import tqdm

steps = [2**n for n in range(10)] + [n * 1000 for n in range(1, 10)] # + [n * 1000 for n in range(10, 143, 5)]
labels = [f"Step {n}" for n in steps]
induction_scores = torch.zeros((model.cfg.n_layers, model.cfg.n_heads, len(steps)), device="cuda")
induction_maxes = torch.zeros(len(steps))
induction_sums = torch.zeros(len(steps))

duplication_scores = torch.zeros((model.cfg.n_layers, model.cfg.n_heads, len(steps)), device="cuda")
duplication_maxes = torch.zeros(len(steps))
duplication_sums = torch.zeros(len(steps))

previous_token_scores = torch.zeros((model.cfg.n_layers, model.cfg.n_heads, len(steps)), device="cuda")
previous_token_maxes = torch.zeros(len(steps))
previous_token_sums = torch.zeros(len(steps))

for i in tqdm(range(len(steps))):
    model = HookedTransformer.from_pretrained(
        model_full_name,
        center_unembed=True,
        center_writing_weights=True,
        fold_ln=True,
        #refactor_factored_attn_matrices=True,
        revision=f"step{steps[i]}",
        cache_dir=cache_dir,
    )

    induction_scores[:,:,i], previous_token_scores[:,:,i], duplication_scores[:,:,i] = get_induction_scores(model)
    induction_maxes[i] = induction_scores[:,:,i].max()
    induction_sums[i] = induction_scores[:,:,i].sum()

  0%|          | 0/19 [00:00<?, ?it/s]

Using pad_token, but it is not set yet.
  5%|▌         | 1/19 [00:03<00:55,  3.08s/it]

Loaded pretrained model EleutherAI/pythia-410m into HookedTransformer


Using pad_token, but it is not set yet.
 11%|█         | 2/19 [00:05<00:49,  2.92s/it]

Loaded pretrained model EleutherAI/pythia-410m into HookedTransformer


...

Loaded pretrained model EleutherAI/pythia-410m into HookedTransformer


Using pad_token, but it is not set yet.
 95%|█████████▍| 18/19 [03:24<00:30, 30.29s/it]

Loaded pretrained model EleutherAI/pythia-410m into HookedTransformer


Using pad_token, but it is not set yet.
100%|██████████| 19/19 [04:04<00:00, 12.85s/it]

Loaded pretrained model EleutherAI/pythia-410m into HookedTransformer


In [12]:
l_line(induction_maxes, hover_name=labels, log_x=True)


In [13]:
l_line(induction_sums, hover_name=labels, log_x=True)

In [14]:
from circuit_utils import visualize_tensor
visualize_tensor(induction_scores.cpu(), labels, zmin=-1, zmax=1)

IntSlider(value=0, description='Slice:', max=18)

Output()

### Pythia 1.4B

In [8]:
model_name = "pythia-1.4b"

model_full_name = f"EleutherAI/{model_name}"

cache_dir="/media/curttigges/project-files/projects/circuits"

In [9]:
model = HookedTransformer.from_pretrained(
    model_full_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    #refactor_factored_attn_matrices=True,
    cache_dir=cache_dir,
)

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-1.4b into HookedTransformer


In [10]:
from tqdm import tqdm

steps = [2**n for n in range(10)] + [n * 1000 for n in range(1, 5)] # + [n * 1000 for n in range(10, 143, 5)]
labels = [f"Step {n}" for n in steps]
induction_scores = torch.zeros((model.cfg.n_layers, model.cfg.n_heads, len(steps)), device="cuda")
induction_maxes = torch.zeros(len(steps))
induction_sums = torch.zeros(len(steps))

duplication_scores = torch.zeros((model.cfg.n_layers, model.cfg.n_heads, len(steps)), device="cuda")
duplication_maxes = torch.zeros(len(steps))
duplication_sums = torch.zeros(len(steps))

previous_token_scores = torch.zeros((model.cfg.n_layers, model.cfg.n_heads, len(steps)), device="cuda")
previous_token_maxes = torch.zeros(len(steps))
previous_token_sums = torch.zeros(len(steps))

for i in tqdm(range(len(steps))):
    model = HookedTransformer.from_pretrained(
        model_full_name,
        center_unembed=True,
        center_writing_weights=True,
        fold_ln=True,
        #refactor_factored_attn_matrices=True,
        revision=f"step{steps[i]}",
        cache_dir=cache_dir,
    )

    induction_scores[:,:,i], previous_token_scores[:,:,i], duplication_scores[:,:,i] = get_induction_scores(model)
    induction_maxes[i] = induction_scores[:,:,i].max()
    induction_sums[i] = induction_scores[:,:,i].sum()

  0%|          | 0/14 [00:00<?, ?it/s]

Using pad_token, but it is not set yet.
  7%|▋         | 1/14 [01:30<19:36, 90.53s/it]

Loaded pretrained model EleutherAI/pythia-1.4b into HookedTransformer


Using pad_token, but it is not set yet.
  7%|▋         | 1/14 [02:53<37:34, 173.41s/it]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_821523/621415723.py:18 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_821523/621415723.py'                        │
│                                                                                                  │
│ /home/curttigges/miniconda3/envs/circuits/lib/python3.8/site-packages/transformer_lens/HookedTra │
│ nsformer.py:891 in from_pretrained                                                               │
│                                                                                                  │
│    888 │   │   if dtype is not None:                                                             │
│    889 │   │   │   model = model.to(dtype)                                                       │
│    890 │   │                                                                                     │
│ ❱  891 │   │   model.load_and_process_state_dict(                                                │
│    892 │   │   │   state_dict,                                                                   │
│    893 │   │   │   fold_ln=fold_ln,                                                              │
│    894 │   │   │   center_writing_weights=center_writing_weights,                                │
│                                                                                                  │
│ /home/curttigges/miniconda3/envs/circuits/lib/python3.8/site-packages/transformer_lens/HookedTra │
│ nsformer.py:1010 in load_and_process_state_dict                                                  │
│                                                                                                  │
│   1007 │   │   │   │   │   │   )                                                                 │
│   1008 │   │   │   │   │   )                                                                     │
│   1009 │   │   │   │   elif k.startswith("blocks"):                                              │
│ ❱ 1010 │   │   │   │   │   state_dict[k] = v.to(                                                 │
│   1011 │   │   │   │   │   │   devices.get_device_for_block_index(                               │
│   1012 │   │   │   │   │   │   │   int(k.split(".")[1]), self.cfg                                │
│   1013 │   │   │   │   │   │   )                                                                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB (GPU 0; 23.67 GiB total capacity; 18.70 GiB 
already allocated; 56.94 MiB free; 18.72 GiB reserved in total by PyTorch) If reserved memory is >> allocated 
memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and 
PYTORCH_CUDA_ALLOC_CONF

In [ ]:
l_line(induction_maxes, hover_name=labels, log_x=True)


In [ ]:
l_line(induction_sums, hover_name=labels, log_x=True)

In [ ]:
from circuit_utils import visualize_tensor
visualize_tensor(induction_scores.cpu(), labels, zmin=-1, zmax=1)

IntSlider(value=0, description='Slice:', max=18)

Output()

### Pythia 2.8B

In [ ]:
model_name = "pythia-160m"

model_full_name = f"EleutherAI/{model_name}"

cache_dir="/media/curttigges/project-files/projects/circuits"

In [ ]:
model = HookedTransformer.from_pretrained(
    model_full_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    #refactor_factored_attn_matrices=True,
    cache_dir=cache_dir,
)

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


In [ ]:
from tqdm import tqdm

steps = [2**n for n in range(10)] + [n * 1000 for n in range(1, 10)] # + [n * 1000 for n in range(10, 143, 5)]
labels = [f"Step {n}" for n in steps]
induction_scores = torch.zeros((model.cfg.n_layers, model.cfg.n_heads, len(steps)), device="cuda")
induction_maxes = torch.zeros(len(steps))
induction_sums = torch.zeros(len(steps))

duplication_scores = torch.zeros((model.cfg.n_layers, model.cfg.n_heads, len(steps)), device="cuda")
duplication_maxes = torch.zeros(len(steps))
duplication_sums = torch.zeros(len(steps))

previous_token_scores = torch.zeros((model.cfg.n_layers, model.cfg.n_heads, len(steps)), device="cuda")
previous_token_maxes = torch.zeros(len(steps))
previous_token_sums = torch.zeros(len(steps))

for i in tqdm(range(len(steps))):
    model = HookedTransformer.from_pretrained(
        model_full_name,
        center_unembed=True,
        center_writing_weights=True,
        fold_ln=True,
        #refactor_factored_attn_matrices=True,
        revision=f"step{steps[i]}",
        cache_dir=cache_dir,
    )

    induction_scores[:,:,i], previous_token_scores[:,:,i], duplication_scores[:,:,i] = get_induction_scores(model)
    induction_maxes[i] = induction_scores[:,:,i].max()
    induction_sums[i] = induction_scores[:,:,i].sum()

  0%|          | 0/19 [00:00<?, ?it/s]

Using pad_token, but it is not set yet.
  5%|▌         | 1/19 [00:20<06:17, 21.00s/it]

Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


Using pad_token, but it is not set yet.
 11%|█         | 2/19 [00:44<06:17, 22.19s/it]

Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


...

Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


Using pad_token, but it is not set yet.
 95%|█████████▍| 18/19 [06:26<00:16, 16.40s/it]

Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


Using pad_token, but it is not set yet.
100%|██████████| 19/19 [06:49<00:00, 21.57s/it]

Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


In [ ]:
l_line(induction_maxes, hover_name=labels, log_x=True)


In [ ]:
l_line(induction_sums, hover_name=labels, log_x=True)

In [ ]:
from circuit_utils import visualize_tensor
visualize_tensor(induction_scores.cpu(), labels, zmin=-1, zmax=1)

IntSlider(value=0, description='Slice:', max=18)

Output()

### Pythia 6.9B

In [10]:
model_name = "pythia-6.9b"

model_full_name = f"EleutherAI/{model_name}"

cache_dir="model_cache"

In [11]:
from transformers import AutoModelForCausalLM

def load_model(model_name):
    if model_name == "EleutherAI/pythia-6.9b" or model_name == "EleutherAI/pythia-12b":
        print("Loading in bfloat16 model")
        source_model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir="model_cache").to('cpu').half()
        print("Loading into HookedTransformer")
        model = HookedTransformer.from_pretrained(
            model_name,
            center_unembed=True,
            center_writing_weights=True,
            fold_ln=True,
            refactor_factored_attn_matrices=False,
            hf_model=source_model,
        )
    else:
        model = HookedTransformer.from_pretrained(
            model_name,
            center_unembed=True,
            center_writing_weights=True,
            fold_ln=True,
            refactor_factored_attn_matrices=False
        )
    return model


In [12]:
model = load_model("EleutherAI/pythia-6.9b")
#model.set_use_hook_mlp_in(True)

Loading in bfloat16 model


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using pad_token, but it is not set yet.


Loading into HookedTransformer


RuntimeError: "baddbmm_with_gemm" not implemented for 'Half'

In [ ]:
from tqdm import tqdm

steps = [2**n for n in range(10)] + [n * 1000 for n in range(1, 10)] # + [n * 1000 for n in range(10, 143, 5)]
labels = [f"Step {n}" for n in steps]
induction_scores = torch.zeros((model.cfg.n_layers, model.cfg.n_heads, len(steps)), device="cuda")
induction_maxes = torch.zeros(len(steps))
induction_sums = torch.zeros(len(steps))

duplication_scores = torch.zeros((model.cfg.n_layers, model.cfg.n_heads, len(steps)), device="cuda")
duplication_maxes = torch.zeros(len(steps))
duplication_sums = torch.zeros(len(steps))

previous_token_scores = torch.zeros((model.cfg.n_layers, model.cfg.n_heads, len(steps)), device="cuda")
previous_token_maxes = torch.zeros(len(steps))
previous_token_sums = torch.zeros(len(steps))

for i in tqdm(range(len(steps))):
    model = HookedTransformer.from_pretrained(
        model_full_name,
        center_unembed=True,
        center_writing_weights=True,
        fold_ln=True,
        #refactor_factored_attn_matrices=True,
        revision=f"step{steps[i]}",
        cache_dir=cache_dir,
    )

    induction_scores[:,:,i], previous_token_scores[:,:,i], duplication_scores[:,:,i] = get_induction_scores(model)
    induction_maxes[i] = induction_scores[:,:,i].max()
    induction_sums[i] = induction_scores[:,:,i].sum()

  0%|          | 0/19 [00:00<?, ?it/s]

Using pad_token, but it is not set yet.
  5%|▌         | 1/19 [00:20<06:17, 21.00s/it]

Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


Using pad_token, but it is not set yet.
 11%|█         | 2/19 [00:44<06:17, 22.19s/it]

Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


...

Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


Using pad_token, but it is not set yet.
 95%|█████████▍| 18/19 [06:26<00:16, 16.40s/it]

Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


Using pad_token, but it is not set yet.
100%|██████████| 19/19 [06:49<00:00, 21.57s/it]

Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer


In [ ]:
l_line(induction_maxes, hover_name=labels, log_x=True)


In [ ]:
l_line(induction_sums, hover_name=labels, log_x=True)

In [ ]:
from circuit_utils import visualize_tensor
visualize_tensor(induction_scores.cpu(), labels, zmin=-1, zmax=1)

IntSlider(value=0, description='Slice:', max=18)

Output()